# Introduction

## Dataset

European Parliament Proceedings Parallel Corpus is a text dataset used for evaluating language detection engines. The 1.5GB corpus includes 21 languages spoken in EU.  

## Implementation

Implementation is based on [1]

### The reason why this model has been chosen are:

- __Dataset size__

Although it's been shown in the past that simple Naive Bayes (NB) based models are performing well (with test set accuracy around 94%), Scikit naive bayes implementation would require enormous memory size to fit all n-grams (NB models for this task perform well when fed with n-grams of size 1 to 6-grams) from all languages in this 1.5GB corpus. It's been shown that Deep Learning based models can give similar or better results when e.g. Convolutional Neural Networks with 1-dimensional convolutions can "act as" n-grams [2].

- __State-of-art results__

As authors of [1] claim, model "performs well in monolingual and multilingual language identification tasks on six datasets covering 131 languages. The method keeps the accuracy also for short documents and across domains, so it is ideal for off-the-shelf use without preparation of training data."

## Previous results for Language detection task

The highest accuracy the model [1] achieves is 97.7%, but it can distinguish between 131 languages and was trained on much more diversified corpuses (e.g. Wikipedia, newspaper texts or SMS messages sent after Haitian earthquake). 

The European Parliament Proceedings corpus has:
- only 21 languages, 
- most of them are easily distinguishable from each other,
- corpus uses official language and is much less diversified.

Our trained model is expected to achieve better accuracy than reported in [1].

## Model summary

Model is fairly simple: 

- Main part of the model is bi-directional Recurrent Neural Network (RNN) 

- On model input, sentence (as a sequence of characters) is being fed and each character is transformed into character embedding

- Character embeddings are fed into RNN to produce on output single sentence-vector

- Later, sentence-vector is being fed through dense layer with softmax activation function to give on output class probability representing one of the 21 languages

In other words, the model is close to encoding part of the model used for Neural Machine Translation as e.g in [3], where the encoding part builds a vector that is used by decoding part to generate the translation.
In case of Language identification task, decoding part is reduced to single dense layer.

Choice of hyperparameters: Implementation uses a similar set of hyperparameters as in original paper [1] model.

# Data preprocessing

Because of the dataset size and only few simple steps required for this dataset preprocessing, it was done using bash language. 

After downloading ( [download link](http://www.statmt.org/europarl/) ) and extracting the dataset, all files are being put into separate folders. 
For example, all French sentences (located in 9453 text files) were extracted to txt/fr/ subfolder.

Process has been divided into 3 files which should be copied inside main dataset folder.


## 1. File: make_clean_sentences.sh

Performs dataset cleanup and puts them in separate file per language. 

Pipeline details:

- Script takes all files from language folders "txt/{language_code}/*"

- Splits sentences using sentence splitter perl script which is part of the dataset (perl script: split_sentences.perl)

- Removes all sentences that start with "<" or "(" because these are meta tags that shouldn't be part of training data

- Removes all sentences shorter than 6 characters

- Removes text in brackets e.g. "This is sentence (text in brackets)." will become "This is sentence ."

- Just in case there are any duplicated sentences which could produce bias in the model, they are being removed

- Shuffles all rows

- Saves to clean_sentences folder, each language in separate file. Example: all French sentences are being put in file ```clean_sentences/sent_fr.txt```


## 2. File: train_test_split.sh

Takes all sent_*.txt files (each file contains sentences for 1 language) from previous step splits them and into separate validation, test and train files. 



### Important notice: Why separate test set? 
It's been noticed that test dataset provided in Challenge is part of original train dataset. The lack of time caused that instead of removing test sentences from train dataset, 10% of original dataset has been used to generate separate validation file containing sentences that trained model will never see during traning. This file has been used for final accuracy check.

__Split ratio: train 80%, validation 10%, test 10%__


## 3. File: generate_data_file.sh

#### Final dataset files

Each of train/test/validation files is being concatenated into single temporary file which is then shuffled and copied to final output file:
- file ```all_test_data_shuffled.txt``` contains all test sentences 
- file ```all_train_data_shuffled.txt``` contains all train sentences
- file ```all_val_data_shuffled.txt``` contains all validation sentences

Each generated line consist of label and sentence in following format:

    {language code}||{sentence}

Example:

    fi||Ymmärrän huolen korruptiosta ja järjestäytyneestä rikollisuudesta.


#### Debugging dataset 

Script also generates small debugging subset (first 500 lines from 3 files above) and stores them into:
- all_test_data_shuffled_debug.txt
- all_train_data_shuffled_debug.txt
- all_val_data_shuffled_debug.txt

Debugging subset is being used to make quick runs over model to check for bugs during training/validation/test.

### Notice

Original implementation was using Python 3.7, but later it's been discovered that 
both libraries used (TensorFlow and Keras) have unresolved bug (https://github.com/tensorflow/tensorflow/issues/6683) related to model saving/restoring from file.

When Python 3 is used, weights in RNN layers aren't properly restored. Switching to Python 2.7 resolves the problem.

## Generating character based encoding dictionaries

Because RNN model is character-based, we need to generate encoding/decoding dictionaries 
which are later going to be used to build to generate embeddings.

In [5]:
import os

data_folder = '/mnt/disk/datasets/europarl/clean_sentences/'

files = os.listdir(data_folder)
sentences_files = [f for f in files if 'sent' in f]
all_chars = set()

for filename in sentences_files:
    with open(data_folder + filename, 'r') as f:
        #print('reading %s' % f.name)
        for sentence in f:
            #all_chars.update(set(sentence)) # python 3
            all_chars.update(set(sentence.decode("utf-8"))) # python 2

### Number of unique characters

In [18]:
len(all_chars)

489

### All unique characters

In [19]:
print("|".join(sorted(all_chars)))


| |!|"|#|$|%|&|'|)|*|+|,|-|.|/|0|1|2|3|4|5|6|7|8|9|:|;|<|=|>|?|@|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|[|\|]|^|_|`|a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z|{|||}|~|¡|¢|£|¤|§|¨|©|ª|«|¬|­|®|¯|°|±|²|³|´|µ|·|¸|¹|º|»|¼|½|¾|¿|À|Á|Â|Ã|Ä|Å|Æ|Ç|È|É|Ê|Ë|Ì|Í|Î|Ï|Ð|Ñ|Ò|Ó|Ô|Õ|Ö|×|Ø|Ù|Ú|Ü|Ý|Þ|ß|à|á|â|ã|ä|å|æ|ç|è|é|ê|ë|ì|í|î|ï|ð|ñ|ò|ó|ô|õ|ö|÷|ø|ù|ú|û|ü|ý|þ|ÿ|Ā|ā|Ă|ă|Ą|ą|Ć|ć|ĉ|Č|č|Ď|ď|Đ|đ|Ē|ē|Ė|ė|ę|Ě|ě|ğ|ġ|Ģ|ģ|Ī|ī|Į|į|İ|ı|Ĳ|ĳ|Ķ|ķ|Ĺ|ĺ|Ļ|ļ|Ľ|ľ|ŀ|Ł|ł|ń|Ņ|ņ|Ň|ň|ō|ŏ|Ő|ő|Œ|œ|ŕ|ŗ|Ř|ř|Ś|ś|Ŝ|Ş|ş|Š|š|Ţ|ţ|Ť|ť|Ū|ū|ŭ|ů|Ű|ű|ų|ŵ|Ÿ|Ź|ź|Ż|ż|Ž|ž|ǎ|ǐ|ǔ|Ǻ|Ȋ|ȋ|Ș|ș|Ț|ț|ə|ʺ|ʻ|ˇ|ˑ|˗|˚|˛|ˮ|̒|̛|ͺ|΄|Ά|Έ|Ή|Ί|Ό|Ύ|Ώ|ΐ|Α|Β|Γ|Δ|Ε|Ζ|Η|Θ|Ι|Κ|Λ|Μ|Ν|Ξ|Ο|Π|Ρ|Σ|Τ|Υ|Φ|Χ|Ψ|Ω|Ϊ|ά|έ|ή|ί|ΰ|α|β|γ|δ|ε|ζ|η|θ|ι|κ|λ|μ|ν|ξ|ο|π|ρ|ς|σ|τ|υ|φ|χ|ψ|ω|ϊ|ϋ|ό|ύ|ώ|Є|І|А|Б|В|Г|Д|Е|Ж|З|И|Й|К|Л|М|Н|О|П|Р|С|Т|У|Ф|Х|Ц|Ч|Ш|Щ|Ъ|Ю|Я|а|б|в|г|д|е|ж|з|и|й|к|л|м|н|о|п|р|с|т|у|ф|х|ц|ч|ш|щ|ъ|ы|ь|ю|я|ё|є|і|ї|љ|ћ|ѝ|ӕ|ۭ|ܧ|ᵉ|ᵐ|Ḩ|ḩ|ἐ|ἑ|ἢ|ἰ|ὐ|ὰ|ὶ|ᾶ|ῃ|ῆ|ῖ|ῦ|ῶ|​|–|—|―|‘|’|‚|“|”|„|‟|•|…|‪|‬|‭|‮|‰|″|⁄|₂|ₒ|€|℃|№|Ω|⅔|−|∙|─|○|眠|脘|舃|舉|舣|�


### Saving dictionaries for later use

In [20]:
pickle.dump(all_chars, open('output/all_chars_python27.pickle', 'wb'))

## Model details

### Choice of hyperparameters

- input max length = 200
- rnn hidden size = 500
- embedding length = 200
- dropout rate = 0.5
- learning rate = 1e-4
- batch size = 64
- train epochs = 5

Optimizer used: Adam 

Loss: sparse categorical crossentropy    

Most important part of implementation are following lines of code representing neural network model (built using Keras library)

In [10]:
model_input = Input(shape=(input_max_len,))
embedding = Embedding(num_embeddings, 
                      output_dim=embedding_len, 
                      mask_zero=True)(model_input)
rnn = Bidirectional(LSTM(rnn_hidden_size))(embedding)
rnn = Dropout(dropout_rate)(rnn)
output = Dense(num_classes, activation='softmax')(rnn)

## Moving code from notebook to .py files

Because of time required to train model and jupter notebook timeout (disconecting from remote server after few minutes of inactivity) decision was made to move code from notebook to separate python files.

Project files:
- main.py - main project file used to run training and final inference test
- eu_dataset.py - dataset generator for both train and test datasets
- model.py - model implementation together with training and test steps loops
- utils.py - few helper methods

# Training

In order to start training, go to the main project folder and run:

    python main.py --data-folder=/path/to/your/dataset/files --output-folder={log_output_folder}

Parameter details:

- ```--output-folder``` (required) is location where training output log will be stored

- ```--data-folder``` expects full path to folder where your 3 training/test/validation files (all_test_data_shuffled.txt, all_train_data_shuffled.txt and all_val_data_shuffled.txt) are located

### Training time

GTX 1070 was used as a training GPU. Each 3000 batches (batch size 64) mini validation check was performed on test set using 200 batches (12.800 sentences). After 160.000 training batches (which is not even one full epoch) model training was stopped because it already reached satisfying results on "mini validation" test set.

Result on "mini validation" test set:
- loss: 0.0103319
- accuracy: 0.9975

Training 160.000 batches on GPU took 28 hours.

Each 10.000 batches model was being saved to checkpoint file, so that full test set check could be performed at a later stage.

#### Training log file

- Every 100 batches outputs batch number, train loss and accuracy, example: ```Batch: 163800/351542, Batch train loss: 0.0116349, batch train accuracy: 0.984375```
    
- Every 3000 batches - performs "mini validation" check on 200 batches from validation file, example: ```19:53:17, Validation checkpoint. Loss: 0.0105691, val acc: 0.997734```
    
    
- Every 10000 batches - saves them model in format: ```model_step_[STEP_NUMBER]_val_acc_[ACCURACY]``` (example: ```model_step_120000_val_acc_0.996484```) and outputs info to log file: 
```17:47:22, Saving model. Val loss: 0.0115113, val acc: 0.997109```

# Results

As already mentioned, test dataset provided in Challenge is part of original train dataset. Using this file would give incorrect inference results. Therefore final test was performed on test dataset generated in preprocessing phase (see section "Dataset preprocessing" above). This test dataset contains sentences that model never saw during training. 

### Loading saved model and checking accuracy on test set

Download pre-trained model: https://www.dropbox.com/s/e7m3wj00sbciz24/model_step_160000.tar.gz

Archive contains 3 files with file extensions: ```.data-00000-of-00001```, ```.index``` and ```.meta```. Unzip it and copy to e.g. ```model``` subfolder.

In order to validate trainied model execute:

    python main.py --output-folder=/path/to/your/dataset/files --data-folder={log_output_folder}
                   --validate=yes --saved-model={model_path_and_filename} --batch-size=500
                   
Parameter details:

- ```--output-folder``` (required) is location where training output log will be stored

- ```--data-folder``` expects full path to folder where your 3 training/test/validation files (all_test_data_shuffled.txt, all_train_data_shuffled.txt and all_val_data_shuffled.txt) are located

- ```--saved-model``` model location path ; full path and model filename is expected, e.g. for filename ```model_step_120000_val_acc_0.996484.meta``` located in subfolder ```models``` it should be ```--saved-model=./models/model_step_120000_val_acc_0.996484``` (notice relative location ```./``` and removed extension)

- ```--validate=yes``` runs validation on test set instead of model training

- ```--batch-size``` during validation batch size can have much higher value to speed up inference (lower the value if you're getting memory error)                   

Output from the validation set (with Tensorflow warnings removed) run on 5624 batches, each batch of size 500 (which gives around 2.812.000 sentences) 

````
Performing final validation on 5624 batches
Final validation loss: 0.0106761, accuracy: 99.695%
````

Inference step takes ~2h.

### Conclusion

Model achieves 2% higher accuracy than reported in original paper [1]. Section "Previous results for Language detection task" already lists possible reasons why one might expect better results than in [1].

Another reason might be caused by training and test sets being from exactly the same corpus, using similar words and official speech style. The model would probably reach much lower accuracy when tested on different type of corpora e.g. news or tweets.

---
[1] "LanideNN: Multilingual Language Identification on Character Window" (https://arxiv.org/abs/1701.03338).

[2] Convolutional Neural Networks for Sentence Classificatio (http://emnlp2014.org/papers/pdf/EMNLP2014181.pdf)

[3] "Google's Neural Machine Translation System: Bridging the Gap between Human and Machine Translation" (http://arxiv.org/abs/1609.08144)